In [ ]:
!pip install -q -U langchain-text-splitters langchain-community langgraph langchain-openai langchain-chroma pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""  # insert key here

## Load the data to documents

In [ ]:
!unzip /content/hardware_materials.zip

Archive:  /content/hardware_materials.zip
   creating: materials/
  inflating: materials/.DS_Store     
   creating: materials/subtitles/
  inflating: materials/subtitles/.DS_Store  
  inflating: materials/subtitles/lecture_10_en.srt  
  inflating: materials/subtitles/lecture_11a_en.srt  
  inflating: materials/subtitles/lecture_11b_en.srt  
  inflating: materials/subtitles/lecture_12a_en.srt  
  inflating: materials/subtitles/lecture_12b_en.srt  
  inflating: materials/subtitles/lecture_1a_en.srt  
  inflating: materials/subtitles/lecture_1b_en.srt  
  inflating: materials/subtitles/lecture_2a_en.srt  
  inflating: materials/subtitles/lecture_2b_en.srt  
  inflating: materials/subtitles/lecture_3a_en.srt  
  inflating: materials/subtitles/lecture_3b_en.srt  
  inflating: materials/subtitles/lecture_4a_en.srt  
  inflating: materials/subtitles/lecture_4b_en.srt  
  inflating: materials/subtitles/lecture_5a_en.srt  
  inflating: materials/subtitles/lecture_5b_en.srt  
  inflating: mater

## Creating summeries for each lecture

In [ ]:
import os
import re
from langchain_openai import ChatOpenAI

PATH_TO_SUMMARIES  = '/content/materials/summaries/'
PATH_TO_TEXTS = '/content/materials/texts'

# read file content and return it
def load_file_content(file_name: str):
    try:
        with open(os.path.join(PATH_TO_TEXTS, file_name), "r") as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file {file_name} was not found.")
        return None

# generate summary for given content
def create_summary_content(content: str, lecture_number: int, model: ChatOpenAI):
    prompt = (
        "Act as a teacher who is a professional summarizer and expert in hardware. Write a detailed, comprehensive, structured and easy-to-understand summary of the provided content.\n"
        f"The title should include 'Summary of Lecture {lecture_number}'.\n"
        "Start with a brief overview that explains the main purpose and structure of the lecture in simple terms.\n"
        "Organize the summary into clear sections and with well-structured paragraphs and bullet points based on the main topics discussed.\n"
        "For each section:\n"
        "1. Elaborate thoroughly on explaining the key concepts, themes, or methodologies. "
        "For each subject, explain the subject in depth, including definition and explanation.\n"
        "2. Include specific and concrete example for every key concept.\n"
        "3. Highlight connections between different topics discussed, especially where they build upon each other or showcase cause-and-effect relationships.\n"
        "Make the language professional, concise, and tailored to explain the material clearly to students.\n"
        "Rely strictly on the provided text, without including external information.\n"
        "The goal is to provide a summary that is accurate, easy to follow, and helps students quickly understand and review the material, without omitting any important information.\n"
        f"Content to summarize:\n\n{content}\n"
    )
    response = model.invoke(prompt)
    return response.content

def add_summeries_files(text_files: list, to_create_summeries: bool):
  summary_files = []
  os.makedirs(PATH_TO_SUMMARIES, exist_ok=True)

  # if summeries files are already exists - only add them to the list and return it (the list)
  if not to_create_summeries:
    summary_files = [f for f in os.listdir(PATH_TO_SUMMARIES) if f.endswith('.txt')]
    # check if there are really files there, and return the list
    if len(summary_files) > 0:
      print(f"The existing files names were added to the list")
    else:
      print(f"Did not found existing summary files in {PATH_TO_SUMMARIES}!!")
    return summary_files

  # clear old files from the folder (if they exists)
  for filename in os.listdir(PATH_TO_SUMMARIES):
    file_path = os.path.join(PATH_TO_SUMMARIES, filename)
    if os.path.isfile(file_path):
      os.remove(file_path)

  # create dict of number of lecture and its files
  lectures_dict = {}
  pattern = re.compile(r"lecture_(\d+)[a-z]*_en.txt")
  for file in text_files:
    match = pattern.search(file)
    if match:
        number = match.group(1)
        if number in lectures_dict:
          lectures_dict[number].append(file)
        else:
          lectures_dict[number] = [file]

  # create LLM object
  summerized_llm = ChatOpenAI(model="gpt-4o-mini")

  # for every lecture:
  for lecture_num in lectures_dict:
    # 1. read the files and create one combined content
    lectures_dict[lecture_num].sort()
    lecture_content = ""
    for file_name in lectures_dict[lecture_num]:
      file_content = load_file_content(file_name)
      if file_content is not None:
        lecture_content += file_content + "\n"

    if lecture_content != "":
      # 2. create summary of the content using the llm
      summary_content = create_summary_content(lecture_content, lecture_num, summerized_llm)

      # 3. put the summary into a file
      summary_file_name = f"Summary_of_lecture_{lecture_num}_en.txt"
      with open(os.path.join(PATH_TO_SUMMARIES, summary_file_name), "w") as file:
        file.write(summary_content)

      # 4. enter the file name to text_files
      summary_files.append(summary_file_name)

    print(f"Finished to summerize lecture number {lecture_num}.")

  return summary_files

text_files = [f for f in os.listdir(PATH_TO_TEXTS) if f.endswith('.txt')]
summary_files = add_summeries_files(text_files, to_create_summeries = True)

Finished to summerize lecture number 4.
Finished to summerize lecture number 8.
Finished to summerize lecture number 11.
Finished to summerize lecture number 3.
Finished to summerize lecture number 7.
Finished to summerize lecture number 6.
Finished to summerize lecture number 9.
Finished to summerize lecture number 2.
Finished to summerize lecture number 5.
Finished to summerize lecture number 1.
Finished to summerize lecture number 10.
Finished to summerize lecture number 12.
